In [26]:
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen, urlretrieve
from selenium import webdriver
from tqdm import tqdm
import os, re, time, random
from multiprocessing.dummy import Pool as ThreadPool
import multiprocessing
import pandas as pd

# Analyze Pandas usage in Kaggle Kernels

## 1. Get urls of top kernels by vote

In [2]:
soup = BeautifulSoup(open("site/Kernels _ Kaggle.htm"), "html.parser")

In [3]:
kernel_links = [a['href'] + '/code' for a in soup.find_all("a", class_="block-link__anchor")]

## 2. Get url of code files from kernels page

In [4]:
THREADS = 1
browsers = [webdriver.Chrome('/Users/adithya/Documents/university/research/scrapingKaggle/chromedriver') for _ in range(THREADS)]
pool = ThreadPool(THREADS)
blacklist = ['https://www.kaggle.com/stkbailey/teaching-notebook-for-total-imaging-newbies/code']

In [5]:
def get_code_from_kernel(kernel_link):
    if kernel_link in blacklist:
        return None
    browser = browsers[random.randint(0, THREADS - 1)]
    try:
        browser.get(kernel_link)
        innerHTML = browser.execute_script("return document.body.innerHTML")
        soup = BeautifulSoup(innerHTML, 'html.parser')

        link = soup.find_all("a", class_="script-code-pane__download")[0]['href']

        print("S")
        
        return link
    except Exception as e:
        print("E", e)
        return None

In [6]:
#code_links_write = pool.map(get_code_from_kernel, kernel_links)
code_links_write = [get_code_from_kernel(kl) for kl in kernel_links[:5]]

S
S
S
S
S


In [7]:
code_links_write

['/kernels/scriptcontent/2020416/download',
 '/kernels/scriptcontent/3123055/download',
 '/kernels/scriptcontent/2368078/download',
 '/kernels/scriptcontent/928737/download',
 '/kernels/scriptcontent/3205416/download']

In [11]:
code_file_write = open('code_links.txt', 'w')
for code_link in code_links_write:
    code_file_write.write("%s\n" % code_link)

In [18]:
code_links_read = []
with open('code_links.txt', 'r') as code_file_read:
    code_links_read = code_file_read.read().splitlines()

In [19]:
code_links = ['https://www.kaggle.com' + cl for cl in code_links_read]
code_links[:5]

['https://www.kaggle.com/kernels/scriptcontent/2020416/download',
 'https://www.kaggle.com/kernels/scriptcontent/3123055/download',
 'https://www.kaggle.com/kernels/scriptcontent/2368078/download',
 'https://www.kaggle.com/kernels/scriptcontent/928737/download',
 'https://www.kaggle.com/kernels/scriptcontent/3205416/download']

## 3. Pull raw code file to local dir

In [20]:
for code_link in tqdm(code_links):
    !wget --content-disposition $code_link -P data

  0%|          | 0/5 [00:00<?, ?it/s]

--2018-04-14 11:47:23--  https://www.kaggle.com/kernels/scriptcontent/2020416/download
Resolving www.kaggle.com (www.kaggle.com)... 168.62.224.124
Connecting to www.kaggle.com (www.kaggle.com)|168.62.224.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16932 (17K) [application/octet-stream]
Saving to: ‘data/script.Rmd’

script.Rmd          100%[===================>]  16.54K  --.-KB/s    in 0.06s   

2018-04-14 11:47:25 (298 KB/s) - ‘data/script.Rmd’ saved [16932/16932]



 20%|██        | 1/5 [00:01<00:07,  1.97s/it]

--2018-04-14 11:47:25--  https://www.kaggle.com/kernels/scriptcontent/3123055/download
Resolving www.kaggle.com (www.kaggle.com)... 168.62.224.124
Connecting to www.kaggle.com (www.kaggle.com)|168.62.224.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48677 (48K) [application/octet-stream]
Saving to: ‘data/script.ipynb’

script.ipynb        100%[===================>]  47.54K  --.-KB/s    in 0.06s   

2018-04-14 11:47:26 (754 KB/s) - ‘data/script.ipynb’ saved [48677/48677]



 40%|████      | 2/5 [00:02<00:03,  1.24s/it]

...

 60%|██████    | 3/5 [00:03<00:02,  1.14s/it]

--2018-04-14 11:47:27--  https://www.kaggle.com/kernels/scriptcontent/928737/download
Resolving www.kaggle.com (www.kaggle.com)... 168.62.224.124
Connecting to www.kaggle.com (www.kaggle.com)|168.62.224.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23763 (23K) [application/octet-stream]
Saving to: ‘data/script.ipynb.2’

script.ipynb.2      100%[===================>]  23.21K  --.-KB/s    in 0.06s   

2018-04-14 11:47:27 (402 KB/s) - ‘data/script.ipynb.2’ saved [23763/23763]



 80%|████████  | 4/5 [00:03<00:00,  1.02it/s]

--2018-04-14 11:47:27--  https://www.kaggle.com/kernels/scriptcontent/3205416/download
Resolving www.kaggle.com (www.kaggle.com)... 168.62.224.124
Connecting to www.kaggle.com (www.kaggle.com)|168.62.224.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60456 (59K) [application/octet-stream]
Saving to: ‘data/script.ipynb.3’

script.ipynb.3      100%[===================>]  59.04K  --.-KB/s    in 0.1s    

2018-04-14 11:47:28 (511 KB/s) - ‘data/script.ipynb.3’ saved [60456/60456]



100%|██████████| 5/5 [00:04<00:00,  1.10it/s]


## 4. Clean non-Python files from downloaded

In [21]:
for filename in os.listdir("data"):
    if filename == ".DS_Store":
        pass
    
    name_parts = filename.split(".")
    if len(name_parts) == 2:
        name_parts.append("0")
        
    if name_parts[1] in ['ipynb', 'py']:
        new_name = name_parts[0] + name_parts[2] + "." + name_parts[1]
        new_name = 'r' + new_name if name_parts[1] == 'py' else new_name
        os.rename('data/' + filename, 'data/' + new_name)

    else:
        os.remove('data/' + filename)

## Tokenize Notebooks and Scripts

In [22]:
# Analyze python script
def parse_script(script_name):
    with open(script_name) as script_file:
        code = script_file.read().splitlines()

    ps = []
    for line in code:
        p = [word for word in line.split(" ") if word.startswith("pd")]
        ps += p

    cleaned_ps = []
    for occ in ps:
        if occ != 'pd' and '(' in occ:
            cleaned_ps.append(occ[:occ.index('(')])
            
    # (TODO) Add modules from import
    return cleaned_ps

In [23]:
all_methods = []
for filename in os.listdir("data"):
    if filename.split(".")[1] == 'py':
        all_methods += parse_script('data/' + filename)
    else:
        pfilename = 'data/' + filename
        cfilename = filename.split(".")[0] + '.py'
        !jupyter nbconvert --to=python $pfilename
        time.sleep(1)
        try:
            all_methods += parse_script('data/' + cfilename)
        except:
            pass

[NbConvertApp] Converting notebook data/script0.ipynb to python
[NbConvertApp] Writing 32874 bytes to data/script0.py
[NbConvertApp] Converting notebook data/script1.ipynb to python
[NbConvertApp] Writing 29921 bytes to data/script1.py
[NbConvertApp] Converting notebook data/script2.ipynb to python
[NbConvertApp] Writing 14727 bytes to data/script2.py
[NbConvertApp] Converting notebook data/script3.ipynb to python
[NbConvertApp] Writing 29733 bytes to data/script3.py


In [24]:
methodFreq = {}
methods_write = open('methods.txt', 'w')
for method in all_methods:
    if method not in methodFreq:
        methodFreq[method] = 0
    methodFreq[method] += 1
        
    methods_write.write("%s\n" % method)

In [29]:
mFreq = pd.DataFrame.from_dict(methodFreq, 'index')
mFreq

,0
pd.read_csv,8
pd.concat,9
pd.Series,1
pd.get_dummies,1
pd.qcut,1
pd.cut,1
pd.DataFrame,5
pd.melt,2
pd.to_datetime,2
